In [5]:
import joblib
import sklearn
import numpy as np
import os
import requests
import json
import urllib
import pandas as pd

locations = ['基隆', '彭佳嶼', '國三N002K', '國三S004K', '國三S007K', '國一S001K', '國一S006K', '竹子湖', '鞍部', '臺北', '大安森林', '國三甲005K', '國三S016K', '板橋', '淡水', '新店', '國三S010K', '國三S037K', '國三S042K', '國三N046K', '國三S054K', '國一S026K', '拉拉山', '新屋', '國二E009K', '國三N063K', '國一S072K', '西濱S023K', '西濱S032K', '國三S103K', '西濱S082K', '新竹', '國一S105K', '國三N076K', '國一N077K', '西濱N066K', '國一N142K', '國一S114K', '國一S123K', '國一S132K', '國一S152K', '國三N119K', '國三N151K', '國三S140K', '西濱N107K', '臺中', '梧棲', '武陵', '國一N174K', '國一S162K', '國一S169K', '國一S188K', '國三N208K', '國三S173K', '國三S178K', '西濱S189K',
             '西濱N210K', '彰師大', '田中', '國一S207K', '國三N191K', '日月潭', '玉山', '國三N223K', '國三S217K', '麥寮', '古坑', '國一N234K', '國三N252K', '西濱S241K', '西濱N222K', '國三N295K', '嘉義', '阿里山', '國一N250K', '國一S262K', '國一N268K', '國三N303K', '西濱S267K', '西濱S257K', '南區中心', '永康', '國一N288K', '國一S306K', '國一N316K', '國一N335K', '國三S311K', '國三S329K', '國三S342K', '國三S366K', '西濱S280K', '西濱N304K', '高雄', '東沙島', '國一S342K', '國一S346K', '國一N351K', '國一N361K', '國三N372K', '國三S385K', '恆春', '國三N425K', '宜蘭', '蘇澳', '花蓮', '合歡山', '成功', '臺東', '大武', '蘭嶼', '吉貝', '東吉島', '澎湖', '金門(東)', '金門', '九宮碼頭', '東引', '馬祖']



mtext = '國三S385K'

if mtext in locations:
    locationName = mtext
    try:
        df_get = get(locationName)  # 獲得天氣data
#         message = TextSendMessage(
#             text= f'現在氣溫:{df.iloc[0,3]}'
#             )
        if df_get.isna().sum().sum() != 0:  # 判斷是否需要補值
            kNN_imputer = joblib.load('kNN_imputer.joblib')
            df_imputed = kNN_imputer.transform(df_get)
            df_imputed = pd.DataFrame(df_imputed, columns = df_get.columns)
        df_full = merge_typhoon_data(df_imputed)  # 合併
        MMscaler = joblib.load('MMscaler.joblib')
        x = MMscaler.transform(df_full)
        model = joblib.load('rf_model.joblib')
        prediction = model.predict_proba(x)
        dayoff_proba = prediction[0][1]*100
        print('dayoff_proba:', dayoff_proba)
#         if dayoff_proba >= 90:
#             try:
#                 message = f'明天放颱風假機率:{round(dayoff_proba, 1)}%\n超高機率放颱風假，祝您假期愉快！'
#             except:
#                 message = "不好意思~請您再試一次"
#         elif dayoff_proba >= 80:
#             try:
#                 message = f'明天放颱風假機率:{round(dayoff_proba, 1)}%\n高機率放颱風假，請做好準備！'
#             except:
#                 message = "不好意思~請您再試一次"
#         elif dayoff_proba >= 60:
#             try:
#                 message = f'明天放颱風假機率:{round(dayoff_proba, 1)}%\n可以期待一下颱風假！'
#             except:
#                 message = "不好意思~請您再試一次"
#         elif dayoff_proba >= 40:
#             try:
#                 message = f'明天放颱風假機率:{round(dayoff_proba, 1)}%\n預言大師也算不準明天到底會不會放颱風假...'
#             except:
#                 message = "不好意思~請您再試一次"
#         elif dayoff_proba >= 20:
#             try:
#                 message = f'明天放颱風假機率:{round(dayoff_proba, 1)}%\n明天不太可能放颱風假哦！'
#             except:
#                 message = "不好意思~請您再試一次"
#         else:
#             try:
#                 message = f'明天放颱風假機率:{round(dayoff_proba, 1)}%\n您還是別妄想颱風假了～～'
#             except:
#                 message ="不好意思~請您再試一次"
#         print(message)
    except:
        print('再試一次')
else:
    print(f'{mtext} not in locations')


def get(locationName):
    url = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0003-001?'
    params = {
        'Authorization': 'CWB-5393733F-1F8E-4BD2-A358-0360AABEE6EB',
        'format': 'JSON',
        'locationName': locationName,
    }

    url = url + urllib.parse.urlencode(params)
    data = requests.get(url).json()
    stnInfo = data['records']['location'][0]
    weatherElement = data['records']['location'][0]['weatherElement']
    obs = {
        'lat': [float(stnInfo['lat'])],
        'lon': [float(stnInfo['lon'])],
    }
    name_mapping = {
        'lat': 'lat',
        'lon': 'lon',
        'Dayoff': 'Dayoff',
        'ELEV': 'StnHeight',
        'HUMD': 'RH',  ######## 沒有*100
        'D_TX': 'T.Max',
        'D_TN': 'T.Min',
        'TEMP': 'Temperature',
        'PRES': 'StnPres',  ######## -99要補NA
        '24R': 'Precp',
        'WD_vector_x': 'WD_vector_x',
        'WD_vector_y': 'WD_vector_y',
        'WDGust_vector_x': 'WDGust_vector_x',
        'WDGust_vector_y': 'WDGust_vector_y',
        'WDSD': 'A',
        'WDIR': 'B',
        'H_XD': 'C',
        'H_FX': 'D',

    }
    for i in range(len(weatherElement)):  ###########
        name = weatherElement[i]['elementName']
        value = weatherElement[i]['elementValue']
        if name in name_mapping.keys():
            obs[name] = [float(value)]
            if float(value) < -90:
                obs[name] = [np.nan]
            else:
                if name == 'HUMD':
                    v = float(value)*100
                    obs[name] = [v]

        # 處理wind
        # 轉換 WS, WG, WSGust, WDGust
        # 氣象角度轉為及座標單位向量公式：(-theta + 90) * pi/180，再分別以cos, sin處理得到單位向量
        # 風速x單位向量
    try:
        WD = obs['WDIR'][0]
        WS = obs['WDSD'][0]
        WD_unit_vector_x = round(np.cos((-WD + 90) * np.pi/180), 5)
        WD_unit_vector_y = round(np.sin((-WD + 90) * np.pi/180), 5)
        WD_vector_x = np.sqrt(WS) * WD_unit_vector_x
        WD_vector_y = np.sqrt(WS) * WD_unit_vector_y
    except:
        WD_vector_x = np.nan
        WD_vector_y = np.nan

    try:
        WDGust = obs['H_XD'][0]
        WSGust = obs['H_FX'][0]
        WDGust_unit_vector_x = round(np.cos((-WDGust + 90) * np.pi/180), 5)
        WDGust_unit_vector_y = round(np.sin((-WDGust + 90) * np.pi/180), 5)
        WDGust_vector_x = np.sqrt(WSGust) * WDGust_unit_vector_x
        WDGust_vector_y = np.sqrt(WSGust) * WDGust_unit_vector_y
    except:
        WDGust_vector_x = np.nan
        WDGust_vector_y = np.nan

    obs['WD_vector_x'] = WD_vector_x
    obs['WD_vector_y'] = WD_vector_y
    obs['WDGust_vector_x'] = WDGust_vector_x
    obs['WDGust_vector_y'] = WDGust_vector_y
    
    df = pd.DataFrame(obs)
    df = df.rename(name_mapping, axis=1)
    df = df.drop(['A', 'B', 'C', 'D', ], axis=1)
    df = df.reindex(sorted(df.columns), axis = 1)
    return df


# def kNN_imputation(df):
#     # kNN補值，若沒有NA則
#     kNN_imputer = joblib.load(os.getcwd()+'/models/kNN_imputer.joblib')
#     df = kNN_impoter.fit_transform(df)
#     return df


def merge_typhoon_data(df_get):
 #    颱風的資料寫死在這裡，選的是莫拉克颱風
  #  1. 字典形式列出特徵工程後還存在的颱風參數
 #   2. 合併抓下來的觀測資料與颱風資料
 #   3. 回傳含有颱風&觀測資料的dataframe
    # 1.
    typhoon_feature = {
        'Dayoff': [1.],  # 假設放假
        'route_3': [1.],
        'route_2': [1.],
        'route_--': [1.],
        'hpa': [955.],
        'TyWS': [40.],
        'X7_radius': [250.],
        'X10_radius': [100.],
        'alert_num': [36.],
        'born_spotE': [136.],
        'born_spotN': [21.],
    }

    df_typhoon = pd.DataFrame(typhoon_feature)
    df_get = df_get.join(df_typhoon, how='left')
    df_get = df_get.reindex(sorted(df_get.columns), axis=1)
    return df_get


# def MM_scale(df):
#     # 使用fit training set的MinMaxScaler壓縮data到0~1之間
#     MMscaler = joblib.load(os.getcwd()+'/models/MMscaler.joblib')
#     df_MM = MMscaler.transform(df)

#     return df_MM


# def prediction(x):
#     # 使用model進行預測
#     # 要改成mtext回覆的格式
#     model = joblib.load(os.getcwd()+'/models/rf_model.joblib')
#     prediction = model.predict_proba(x)
#     dayoff_proba = prediction[0][1]*100
#     return dayoff_proba




ok
get ok
需要補值
dayoff_proba: 49.984701788084166


/Users/shyanechang/opt/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [8]:
df_get

,Precp,RH,StnHeight,StnPres,T.Max,T.Min,Temperature,WDGust_vector_x,WDGust_vector_y,WD_vector_x,WD_vector_y,lat,lon
0,0.0,82.0,93.0,NaN,19.8,13.6,19.8,0.245578,1.392732,-0.353553,0.612376,22.769551,120.436379


In [8]:
for col in df_full.columns:
    if df_full.loc[0,col] <= -90:
        df_full.loc[0,col] = np.nan

NameError: name 'df_full' is not defined

In [7]:
x_full = pd.DataFrame(x, columns = df_full.columns)
pd.concat([df_full.T,x_full.T], axis = 1)

,0,0
Dayoff,1.000000,1.000000
Precp,0.000000,0.000000
RH,82.000000,0.625000
StnHeight,93.000000,0.037038
StnPres,998.600000,0.967825
T.Max,19.800000,0.403458
T.Min,13.600000,0.410714
Temperature,19.800000,0.454545
TyWS,40.000000,0.372881
WDGust_vector_x,0.245578,0.528218


In [6]:
df.columns

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [34]:
value = '0.3'
float(value)*100

30.0